In [1]:
import os 
import sys

sys.path.append('/home/kalkiek/projects/reddit-political-affiliation/')

import itertools
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from tqdm.notebook import tqdm

from src.data.make_dataset import build_dataset
from src.models.word2vec.User2Subreddit import User2Subreddit
# from src.models.word2vec.predict_model import predict_user_affiliations, top_n_similar_embeddings, \
#     save_similar_embeddings_to_tsv

from sklearn.metrics import auc, roc_curve

# Testing/Validating the Model

### Load in the datset

In [2]:
year_month = '2019-01'

network_path = '/shared/0/projects/reddit-political-affiliation/data/bipartite-networks/' + year_month + '_filtered.tsv'
flair_directory =  '/shared/0/projects/reddit-political-affiliation/data/flair-affiliations/' + year_month + '.tsv'

dataset, training, validation, pol_validation, vocab = build_dataset(network_path, flair_directory)


Building vocab from file:  23%|██▎       | 6053501/26729161 [00:12<00:35, 580846.09it/s]


Building vocab from file:  44%|████▍     | 11839167/26729161 [00:25<00:27, 549548.73it/s]


Building vocab from file:  65%|██████▌   | 17377460/26729161 [00:37<00:15, 586046.62it/s]


Building vocab from file:  85%|████████▍ | 22640214/26729161 [00:52<00:10, 392107.84it/s]


Converting data to PyTorch:   0%|          | 161/5120865 [00:00<53:03, 1608.72it/s]

Length of vocab: 5178863
User count: 5120865
Subreddit count: 57998
User to politic counts: 2633
[('Cord_inate8', Counter({'Republican': 2})), ('error404brain', Counter({'Democrat': 1})), ('OTIS_is_king', Counter({'Democrat': 1})), ('Nesano', Counter({'Republican': 1})), ('SinisterPaige', Counter({'Republican': 1})), ('grubas', Counter({'Republican': 1})), ('DrPiccoloPhD', Counter({'Republican': 1})), ('Hillarys_cellmate', Counter({'Republican': 2})), ('guanaco55', Counter({'Republican': 2})), ('nycola', Counter({'Democrat': 1}))]
Saw political affiliations for 2632 users
User to politics training size: {}: 2369
User to politics validation size: {}: 263


Converting data to PyTorch: 100%|██████████| 5120865/5120865 [08:26<00:00, 10115.12it/s]


Train size: 144337470 Validation size: 16037496


In [7]:
# We'll need these later
word_to_ix = {word: i for i, word in enumerate(vocab)}
all_subreddits = {v for v in vocab if v[:2] == 'r/' and v[2:4] != 'u_'}
print("# of subreddits: " + str(len(all_subreddits)))

# of subreddits: 55204


### Load in the model

In [3]:
PATH = '/shared/0/projects/reddit-political-affiliation/working-dir/word2vec-outputs/' + year_month + '/9.pt'
embedding_dim = 50

# Sorry for the hardcoding ... will update later
model = User2Subreddit(5138256, embedding_dim, 80881)
model.load_state_dict(torch.load(PATH))
model.eval()

User2Subreddit(
  (u_embeddings): Embedding(5138256, 50)
  (v_embeddings): Embedding(80881, 50)
  (political_layer): Linear(in_features=50, out_features=1, bias=True)
  (before_pol_dropout): Dropout(p=0.5, inplace=False)
)

### Predict on the validation set

In [4]:
user_ids, pol_labels = [], []

for user, pol_label in pol_validation.items():
    try:
        # User subreddit dataset spans 1 month. Political data spans the year. Some users might not be present
        user_ids.append(dataset.user_to_idx[user])
        pol_labels.append(pol_label)
    except KeyError:
        pass

user_ids = torch.LongTensor(user_ids)
pol_labels = torch.FloatTensor(pol_labels)

emb_p = model.u_embeddings(user_ids)
political_predictions = model.political_layer(emb_p)
political_predictions = torch.sigmoid(political_predictions)

In [5]:
# Quick clean up

preds = []
for val in political_predictions.detach().numpy():
    if val[0] >= 0.5:
        preds.append(1)
    else:
        preds.append(0)

labels = pol_labels.detach().numpy().astype(int)

In [6]:
from sklearn.metrics import accuracy_score


accuracy_score(labels, preds)

0.7735042735042735

### Similar Embeddings 

In [15]:
sample_subreddits = ['r/nba', 'r/CryptoCurrency', 'r/Conservative', 'r/Liberal', 'r/AskReddit',
                     'r/Aww', 'r/Games', 'r/Hunting', 'r/Feminism', 'r/The_Donald',
                     'r/lawnmowers', 'r/juul', 'r/teenagers']

def top_n_similar_embeddings(model, subreddit, all_subreddits, word_to_ix, n):
    cosine_sims = {}
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)

    print('looking up ', subreddit)

    try:
        sub_tensor = torch.tensor([word_to_ix[subreddit]], dtype=torch.long)
    except KeyError:
        return
    count = 0
    for sub in all_subreddits:
        ix = word_to_ix[sub]
        lookup_tensor = torch.tensor([ix], dtype=torch.long)
        try :
            cos_result = cos(model.u_embeddings(sub_tensor), model.u_embeddings(lookup_tensor))
            cosine_sims[sub] = cos_result
        except Exception:
            count += 1
    print(count)        

    # Sort and grab the top N
    cosine_sims = {k: v for k, v in sorted(cosine_sims.items(), key=lambda item: item[1], reverse=True)}
    top_results = dict(itertools.islice(cosine_sims.items(), n))

    # Spit out the results to the console
    print("Top {} similar embeddings for the subreddit: {}".format(n, subreddit))
    for sub, score in top_results.items():
        print(sub, score)

    return top_results


for sample_sub in sample_subreddits:
    top_n_similar_embeddings(model, sample_sub, all_subreddits, word_to_ix, n=10)

looking up  r/nba
453


NameError: name 'itertools' is not defined